In [2]:
%load_ext cython

In [11]:
%%cython -a
import numpy as np

def mandelbrot_cython(m, size, iterations):
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if np.abs(z) <= 10:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

The **-a** option tells Cython to annotate lines of code with a background color indicating how optimized it is. The darker the color, the less optimized the line. The color depends on the relative number of Python API calls at each line. We can click on any line to see the generated C code. Here, this version does not appear to be optimized.



In [6]:
import numpy as np
size = 400
iterations = 100

s = (size, size)

In [12]:
%%timeit -n1 -r1 m = np.zeros(s, dtype=np.int32)
mandelbrot_cython(m, size, iterations)

6.64 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [19]:
%%cython -a
import numpy as np

def mandelbrot_cython(int[:,::1] m,
                      int size,
                      int iterations):
    cdef int i, j, n
    cdef complex z, c
    for i in range(size):
        for j in range(size):
            c = -2 + 3./size*j + 1j*(1.5-3./size*i)
            z = 0
            for n in range(iterations):
                if z.real**2 + z.imag**2 <= 100:
                    z = z*z + c
                    m[i, j] = n
                else:
                    break

In [20]:
%%timeit -n1 -r1 m = np.zeros(s, dtype=np.int32)
mandelbrot_cython(m, size, iterations)

13.6 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [22]:
m = np.zeros(s, dtype=np.int32)
mandelbrot_cython(m, size, iterations)

This version is almost 350 times faster than the first version!

All we have done is to specify the type of the local variables and function arguments, and bypass NumPy's np.abs() function when computing the absolute value of z. These changes have helped Cython generate more optimized C code from Python code.

如果一行为白色，则表示生成的代码不与 Python 交互，因此将以与普通 C 代码一样快的速度运行。黄色越深，该行中的 Python 交互越多。这些黄色线通常可以在 Python 对象上运行，引发异常，或执行其他类型的高级操作，而不是可以轻松转换为简单快速的 C 代码。函数声明和返回使用 Python 解释器，因此这些行是黄色的。列表推导也是如此，因为它涉及创建 Python 对象。

In [28]:
%%cython -a
# distutils: language = c++

from libcpp.vector cimport vector
def primes_cython(unsigned int nb_primes):
    cdef int n, i
    cdef vector[int] p
    p.reserve(nb_primes)  # allocate memory for 'nb_primes' elements.
    n = 2
    while p.size() < nb_primes:  # size() for vectors is similar to len()
        for i in p:
            if n % i == 0:
                break
        else:
            p.push_back(n)  # push_back is similar to append()
        n += 1
    # Vectors are automatically converted to Python
    # lists when converted to Python objects.
    return p

In [31]:
_=primes_cython(1000)